In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv.gz
/kaggle/input/home-data-for-ml-course/data_description.txt
/kaggle/input/home-data-for-ml-course/test.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv
/kaggle/input/home-data-for-ml-course/test.csv


In [2]:
df_train = pd.read_csv("/kaggle/input/home-data-for-ml-course/train.csv", index_col="Id")
df_test = pd.read_csv("/kaggle/input/home-data-for-ml-course/test.csv", index_col="Id")
df_train["SalePrice"].isnull().any()


False

In [3]:
from sklearn.model_selection import train_test_split


df_train.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = df_train.SalePrice
df_train.drop(['SalePrice'], axis=1, inplace=True)

numerical_data = [i for i in df_train.columns if df_train[i].dtype in ["int64", "float64"]]
categorical_data = [i for i in df_train.columns if df_train[i].dtype == "object" and df_train[i].nunique() < 10]

X = df_train[numerical_data + categorical_data].copy()
X_train, X_valid, y_train, y_valid = train_test_split(X,y,train_size=0.8,
                                                      test_size = 0.2, random_state=0)
X_test = df_test[numerical_data + categorical_data].copy()


In [4]:
X_train.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,Attchd,Unf,TA,TA,Y,NaN,NaN,NaN,New,Partial
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Normal
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Normal
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal


In [5]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

numerical_transformer = SimpleImputer(strategy="constant")
categorical_transformer = Pipeline(steps=[("impute", SimpleImputer(strategy="constant")),
                                         ("onehot", OneHotEncoder(handle_unknown="ignore"))])

preprocessor = ColumnTransformer( transformers=[("num",numerical_transformer, numerical_data),
                                               ("cat", categorical_transformer, categorical_data )])

# model = RandomForestRegressor(n_estimators=300, random_state=0)
model  = XGBRegressor(n_estimators=1000, random_state=0, learning_rate=0.05)
my_pipeline = Pipeline(steps=[("preprocessor", preprocessor),
                             ("model", model)
                             ])



In [6]:
my_pipeline.fit(X, y)

preds = my_pipeline.predict(X_valid)
error = mean_absolute_error(preds, y_valid)
error

545.2228970462329

In [7]:
preds = my_pipeline.predict(X_test)

output = pd.DataFrame({"Id":X_test.index, "SalePrice":preds})
output

,Id,SalePrice
0,1461,128892.164062
1,1462,159863.375000
2,1463,185356.437500
3,1464,191248.390625
4,1465,202431.265625
...,...,...
1454,2915,78746.968750
1455,2916,76918.664062
1456,2917,164407.375000
1457,2918,115062.398438


In [8]:
output.to_csv("submission.csv", index=False)